Installing the necessary libraries through commands prompt.

! -command mode
- edit mode

In [ ]:
!pip install tensorflow
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install keras-resnet 

In order to import/export files from/to local system we have to use 

from google.colab import files

uploaded = files.upload()//files.download()

You can access files in Drive in a number of ways, including:

Mounting your Google Drive in the runtime's virtual machine

Using a wrapper around the API such as PyDrive

Using the native REST API



In [ ]:
from google.colab import files

uploaded = files.upload()

Unzip using the above file using below command.

!unzip hacker.zip

In [ ]:
!unzip hacker.zip

In [ ]:
ls  # list of files in the path

Import Essential libraries needed for Cnn Image classification.

1.import all libraries either from keras.model or tensorflow.keras.model

2.if u combine above these two u will get class instance error

**The added layer must be an instance of class Layer. Found: <tensorflow.python.keras.engine.input_layer.InputLayer>**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,BatchNormalization,Dropout,Activation
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np


from tensorflow.keras.models import load_model
from keras.utils import np_utils
from tensorflow.keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import optimizers
import cv2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


Here we are using Resnet-50 for transfer learning (pre-trained model)


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model


In [ ]:
ls

In [ ]:
train=pd.read_csv('dataset/train.csv') # read .csv file using pandas as dataframe
test=pd.read_csv('dataset/test.csv')

In [ ]:
# !pip install scikit-learn  # installing scikit-learn

In order to convert the output labels to integers and vectors


*   Converting into integers using label encoder

*   then convert integers into vectors using one hot encoder / from keras we can done using to_categorical





```
# This is formatted as code
from sklearn.preprocessing import LabelEncoder
le =LabelEncoder()
le.fit(train.target)
train['target1']=le.transform(train.target)
#X_train,X_valid,y_train,y_valid =train_test_split(train.Image,train.target,test_size=0.1,stratify=train.target)
train['target1']=to_categorical(train['target'])
```



Either we can convert the output label into vectors and feed into into neural network through the training generator - (class_mode -raw,


```
# This is formatted as code
y_col =[columns of vectors of output labels]

columns=['bharatanatyam','kathak','kathakali','kuchipudi','manipuri','mohiniyattam','odissi','sattriya']

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(train[['target']]).toarray())
# merge with main df bridge_df on key values
train = train.join(enc_df)**bold text**

# This is formatted as code
```train_generator=datagen.flow_from_dataframe(
dataframe=train,
directory="dataset/train/",
x_col='Image',
y_col=columns,
subset="training",
batch_size=8,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(224,224))

'''





In [ ]:
!pip install opencv-python

In [ ]:
import os
train_dir='dataset/train/'
X_set = []
IMG_SIZE = 224
for index,row in train.iterrows():
    print(f'\rDone: {os.path.join(train_dir,row[0])}',end='')
    img = cv2.imread(os.path.join(train_dir,row[0]))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    X_set.append(img)

In [ ]:
X = np.array(X_set)
y = train['target'].values

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')


In [ ]:
y = enc.fit_transform(train[['target']]).toarray()

In [ ]:
X = X/255.0

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X[0])
plt.title(f'{y[0]}')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.17, random_state=42)

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,
                             rotation_range=20,
                             zoom_range=0.2,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             shear_range=0.1,
                             fill_mode="nearest")

testgen = ImageDataGenerator()

datagen.fit(X_train)
testgen.fit(X_test)

In [ ]:
IMG_Height=224
IMG_Width=224
input_size=(IMG_Height,IMG_Width,3)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
import tensorflow.keras

inception=InceptionV3(include_top=False,weights='imagenet',input_shape=input_size)
output1=inception.output
output1=GlobalAveragePooling2D()(output1)
output1=Dense(512, activation='relu')(output1)
predictions = Dense(8, activation='softmax')(output1)
inception=Model(inception.input,predictions)
for layers in inception.layers:
  layers.trainable=False
inception.summary()

In [ ]:
'''from tensorflow.keras.layers import GlobalAveragePooling2D
model2 = Sequential()
model2.add(inception)
# model2.add(Conv2D(256,kernel_size=3,padding='same',activation='relu',input_shape=(224,224,3)))
# model2.add(MaxPool2D(pool_size=(2,2)))
# model2.add(Dropout(0.25))
# model2.add(Flatten())
# model2.add(Dense(1024, activation='relu', input_dim=(224,224,3)))
# model2.add(Dense(1024,activation='relu'))
# model2.add(BatchNormalization())
# model2.add(Dropout(0.3))
# model2.add(Dense(512, activation='relu'))
# model2.add(BatchNormalization())
# model2.add(Dropout(0.3))
# model2.add(GlobalAveragePooling2D())
model2.add(Dense(8, activation='softmax')) '''
model2.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])
model2.summary()

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                           patience=3,
                                           verbose=1,
                                           factor=0.5,
                                           min_lr=0.0001)
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
 
cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = 3)
cb_checkpointer = ModelCheckpoint(filepath = '../working/best.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')
 

In [ ]:
callbacks_list=[cb_early_stopper,cb_checkpointer,learning_rate_reduction]

In [ ]:
hist = model2.fit_generator(datagen.flow(X_train,y_train,batch_size=32),
                                        validation_data=testgen.flow(X_test,y_test,batch_size=32),
                                        epochs=50)
                                        # callbacks=callbacks_list)

In [ ]:
#train_labels=train.target.values
le =LabelEncoder()
le.fit(train.target)
train['target2']=le.transform(train.target)
train_Label=to_categorical(train.target2)
train_Label.shape

In [ ]:
train.target1.shape

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(train[['target']]).toarray())
# merge with main df bridge_df on key values
train = train.join(enc_df)
train
#y=train.iloc[:,2:]


In [ ]:
#train.head(10)
train=train.rename({0:'bharatanatyam',1:'kathak',2:'kathakali',3:'kuchipudi',4:'manipuri',5:'mohiniyattam',6:'odissi',7:'sattriya'},axis=1)
train.head()
columns=['bharatanatyam','kathak','kathakali','kuchipudi','manipuri','mohiniyattam','odissi','sattriya']
'''from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(train[['target']]).toarray())
# merge with main df bridge_df on key values
train = train.join(enc_df)
train '''

In order to do image data preprocessing best way to do is 

1.ImageDataGenerator - Data augmentation(train,test,valid data)




```
train_datagen = ImageDataGenerator(
        rescale=1./255,  - normalizing the image b/w 0 to 1
        shear_range=0.2,   - shear transformation
        zoom_range=0.2,     - zooming the image by 20%
        horizontal_flip=True)  - rotating

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)'''

test_datagen = ImageDataGenerator(rescale=1./255)
```



2.flow from

2.1  flow_from_directory -images paths like this
   
main_directory/

  ...class_a/

      ......a_image_1.jpg

      ......a_image_2.jpg

  ...class_b/

    ......b_image_1.jpg

    ......b_image_2.jpg


    

```
train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')
```



*   flow from dataframe  - there is dataframe like train.csv
 
 it contain - Image name -1.jpg,2.jpg...
             
             target  -Kathakali,manipuri...etc

  

```
train_generator=datagen.flow_from_dataframe(
dataframe=train, # dataframe name
directory="dataset/train/", - directory of images located
x_col='Image', - images names in dataframe
y_col=columns,  - target label
subset="training", - 
batch_size=8,
seed=42,
shuffle=True,
class_mode="raw", - "raw" data if output is in vectors ,"categorical" if output is categorical or string. 
target_size=(224,224)) - shape of image of after passing through generator
```


other techniques are also there plz refer keras documentation 
[keras documentation](https://keras.io/api/preprocessing/image/)





In [ ]:
'''train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)'''

test_datagen = ImageDataGenerator(rescale=1./255)

datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.05,shear_range=0.2,
    zoom_range=0.2)


train_generator=datagen.flow_from_dataframe(
dataframe=train,
directory="dataset/train/",
x_col='Image',
y_col='target',
subset="training",
batch_size=8,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

valid_generator=datagen.flow_from_dataframe(
dataframe=train,
directory="dataset/train/",
x_col='Image',
y_col='target',
subset="validation",
batch_size=8,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

#test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
dataframe=test,
directory="dataset/test/",
x_col="Image",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(224,224))


While we are dealing with tranfer learning or pretrained modes like resnet,vgg6,inception..

We will remve the top layer of pretrained model - Fully connected layer  -**include_top=False**

rest of them we will keep as it is ...
1. weights are pretrained except top layer  - **weights='imagenet'**
2. Make all the layers non-trainable (We can retrain some of the lower layers to increase performance. Keep in mind that this may lead to overfitting)


```
for layer in restnet.layers:
    layer.trainable = False
```




```
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output) # giving the resnet-output to keras flatten layer(giving custom output to pretrained model) 
restnet = Model(restnet.input, output=output)
for layer in restnet.layers: 
    layer.trainable = False     # Make all the layers non-trainable (We can retrain some of the lower layers to increase performance. Keep in mind that this may lead to overfitting)

restnet.summary()
```



In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
import keras

inception=InceptionV3(include_top=False,weights='imagenet',input_shape=(224,224,3))
output1=inception.layers[-1].output
output1=keras.layers.Flatten()(output1)
inception=Model(inception.input,output=output1)
for layers in inception.layers:
  layers.trainable=False
inception.summary()


In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output=output)
for layer in restnet.layers:
    layer.trainable = False
restnet.summary()

Custom keras model on the top of pretrained resnet model.

 for binary output loss - binary_crossentropy
 for muli-output   loss -categorical crossentropy
 optimizers -adam,adagrad, rmsprop
 metrics  -accuracy
```
# This is formatted as code



model2.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(learning_rate=1e-3, decay=1e-6,  momentum = 0.9),
              metrics=['accuracy'])

  ```



In [ ]:
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, InputLayer,BatchNormalization
from keras.models import Sequential
from keras import optimizers

model2 = Sequential()
model2.add(inception)
# model2.add(Conv2D(256,kernel_size=3,padding='same',activation='relu',input_shape=(224,224,3)))
# model2.add(MaxPool2D(pool_size=(2,2)))
# model2.add(Dropout(0.25))
# model2.add(Flatten())
model2.add(Dense(1024, activation='relu', input_dim=(224,224,3)))
# model2.add(Dense(1024,activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.3))
model2.add(Dense(512, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.3))
model2.add(Dense(8, activation='softmax'))
model2.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])
model2.summary()

In [ ]:
history = model2.fit_generator(train_generator,
        validation_data = test_generator,
        epochs = 20,
        steps_per_epoch = len(train_generator),
        validation_steps = len(test_generator),
        
)

In [ ]:
history = model2.fit(train_generator,
                               
                              epochs=10,
                              validation_data=valid_generator, 
                      
                              verbose=1)



*   **.fit** is used when the entire training dataset can fit into the memory and no data augmentation is applied.


*   **.fit_generator** is used when either we have a huge dataset to fit into our memory or when data augmentation needs to be applied.



In [ ]:
evaluate =model2.evaluate_generator()

In [ ]:
test_generator.reset()
pred=model2.predict_generator(test_generator,
verbose=1)


In order to view predctions we use this below the code 

generating the predictions file.



In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames
results=pd.DataFrame({"Image":filenames,
                      "target":predictions})
results.to_csv("results.csv",index=False)

In [ ]:
for k in predicted_class_indices:
  print (k, labels[k])

to download the results .csv from collab

In [ ]:
from google.colab import files
files.download('results.csv') 

some pre-checks to stop overfitting the model. -**EarlyStopping, ModelCheckpoint**

Loads the weights of model to save in future to skip training.


```
#model1.load_weights("../working/best.hdf5")

from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = 3)
cb_checkpointer = ModelCheckpoint(filepath = '../working/best.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

fit_history = model1.fit_generator(
        train_generator,
        
        epochs = 10,
        validation_data=validation_generator,
        
        callbacks=[cb_checkpointer, cb_early_stopper]
)
```




To view the performacne metrics like validation accurracy,loss vs epochs 

fit_history is model name 

```
acc=fit_history.history['accuracy']    - modelname.history['accuracy']
val_acc=fit_history.history['val_accuracy']
loss=fit_history.history['loss']
val_loss=fit_history.history['val_loss']

epochs=range(len(acc))

fig = plt.figure(figsize=(20,10))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.show()
fig.savefig('../Accuracy_curve_CNN_256.jpg')
```



In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc))

fig = plt.figure(figsize=(20,10))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.show()
fig.savefig('../Accuracy_curve_CNN_256.jpg')

Custom model for image classification 

In [ ]:
model=Sequential()
model.add(Conv2D(filters=64,kernel_size=3,padding='same',input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=2))

model.add(Conv2D(filters=64,kernel_size=3))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64,kernel_size=3))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2))
#model.add(Dropout(0.25))

model.add(Conv2D(filters=64,kernel_size=3))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(BatchNormalization())
#model.add(Dropout(0.25))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dense(8,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
''' model.fit(train_generator, epochs=10,
                              batch_size=32,
                              verbose=1,
                              validation_data=validation_generator
                            
                              ) '''

checkpoint = ModelCheckpoint('weights.hdf5', monitor='val_loss',
save_best_only=True)
# Store in a list to be used during training
#callbacks_list = [checkpoint]
# Fit the model on a training set, using the checkpoint as a
#callback
history = model.fit_generator(train_generator,
                              epochs=10,
                              
                              verbose=1,
                              validation_data=validation_generator
                            
                              )

In [ ]:
acc=history.history['acc']
val_acc=model.history['val_acc']
loss=model.history['loss']
val_loss=model.history['val_loss']

epochs=range(len(acc))

fig = plt.figure(figsize=(20,10))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.show()
fig.savefig('../Accuracy_curve_CNN_256.jpg')